In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.nn.functional import softmax
from ast import literal_eval
from tqdm import tqdm
from sklearn.metrics import classification_report
tqdm.pandas()
from transformers import AutoTokenizer, AutoModel
import random
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
random.seed(0)
from belt_nlp.bert_with_pooling import BertClassifierWithPooling
from torch.nn.functional import softmax
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import sys
import os
sys.path.append("src/")
from models.classification_methods import create_test_results_df
from data.lambdas import int_to_label, label_to_int


# Define paths
raw_data_path = 'data/raw/'
processed_data_path = 'data/processed/'
reports_path = 'reports/'
file_format_tmt = processed_data_path + "{split}_r3_{target}_top_mentioned_timelines_processed.csv"
file_format_users = processed_data_path + 'r3_{target}_{split}_users_processed.csv' 

In [2]:
# read data
data = pd.read_csv(
    processed_data_path + 'r3_ig_test_users_scored_Timeline.csv',
    sep = ';', 
    encoding='utf-8-sig'
)

In [9]:
eval(data.comments_and_scores_Timeline[1])[-1]

(17,
 'Olha aí @ Segundo pastor evangélico, Pokémon influencia os adolescentes a serem gays — …')